# Analysis of Crime Data in Cuyahoga County

In [4]:
import requests
import json
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
from citipy import citipy
import numpy as np
from scipy.stats import linregress

from api_keys import weather_api_key
from api_keys import fbi_key


In [2]:
#URLs
##API base URL="https://api.usa.gov/crime/fbi/sapi"

##Requests Format: https://api.usa.gov/crime/fbi/sapi/{desired_endpiont}?api_key=<API_KEY>

##See more information about API endpoints at https://crime-data-explorer-master.fr.cloud.gov/api 

#Example Requests:
#query_url=f"https://api.usa.gov/crime/fbi/sapi/api/participation/national?api_key={fbi_key}"
##https://api.usa.gov/crime/fbi/sapi/api/agencies?api_key=iiHnOKfno2Mgkt5AynpvPpUQTEyxE77jo1RU8PIv

In [ ]:
#Make a loop to get the crime data for each agency in Cuyahoga county

##Agency level NIBRS Offender Demographic Endpoint
#https://api.usa.gov/crime/fbi/sapi/api/nibrs/{offense}/offender/agencies/{ori}/{variable}
    #CUYAHOGA county ORI (FIPS=035 UCR=018)
    #ORI numbers reference: https://www.icpsr.umich.edu/files/NACJD/ORIs/39oris.html#39_CUYAHOGA_(FIPS=035_UCR=018)

#Agency level NIBRS Offense Count Endpoint
#https://api.usa.gov/crime/fbi/sapi/api/nibrs/{offense}/offense/agencies/{ori}/{variable} 

#Agency level SRS Crime Data Endpoint Year
#https://api.usa.gov/crime/fbi/sapi//api/summarized/agencies/{ori}/offenses/{since}/{until} 

#Agency level SRS Crime Data Endpoint by Offense
#https://api.usa.gov/crime/fbi/sapi/api/summarized/agencies/{ori}/{offense}/{since}/{until}


#Data summary by population in US
#https://api.usa.gov/crime/fbi/sapi/api/summarized/national/population/{offense}/{since}/{until} 


In [7]:
#Obtain data

In [ ]:
#Json output for each agency data using endpoints above

In [ ]:
#Make a dataframe for each output

In [ ]:
#Clean the data

In [ ]:
#POSSIBLE ENDPOINT REQUEST URLs to use

#State level NIBRS Offense Count Endpoint
#https://api.usa.gov/crime/fbi/sapi/api/nibrs/{offense}/offense/states/{stateAbbr}/{variable}
    
#State level NIBRS Offender Demographic Count Endpoint
#https://api.usa.gov/crime/fbi/sapi/api/nibrs/{offense}/offender/states/{stateAbbr}/{variable} 
    
#State level UCR Estimated Crime Data Endpoint
#https://api.usa.gov/crime/fbi/sapi/api/summarized/estimates/states/{stateAbbr}/{since}/{until}

#Agency level UCR Participation Endpoint
#https://api.usa.gov/crime/fbi/sapi/api/participation/agencies/{ori} 

#Agency level NIBRS Offender Demographic Endpoint
#https://api.usa.gov/crime/fbi/sapi/api/nibrs/{offense}/offender/agencies/{ori}/{variable}
    #CUYAHOGA county ORI (FIPS=035 UCR=018)
    #ORI numbers reference: https://www.icpsr.umich.edu/files/NACJD/ORIs/39oris.html#39_CUYAHOGA_(FIPS=035_UCR=018)

#Agency level NIBRS Offense Count Endpoint
#https://api.usa.gov/crime/fbi/sapi/api/nibrs/{offense}/offense/agencies/{ori}/{variable} 


#Agency level SRS Crime Data Endpoint Year
#https://api.usa.gov/crime/fbi/sapi//api/summarized/agencies/{ori}/offenses/{since}/{until} 

#Agency level SRS Crime Data Endpoint by Offense
#https://api.usa.gov/crime/fbi/sapi/api/summarized/agencies/{ori}/{offense}/{since}/{until}




In [10]:
#State UCR Crime Data for 2018
year_state=f"https://api.usa.gov/crime/fbi/sapi/api/summarized/estimates/states/OH/2018/2018?api_key={fbi_key}"
response=requests.get(year_state).json()
print(json.dumps(response, indent=4, sort_keys=True))

{
    "pagination": {
        "count": 1,
        "page": 0,
        "pages": 1,
        "per_page": 0
    },
    "results": [
        {
            "aggravated_assault": 17674,
            "arson": 1325,
            "burglary": 48186,
            "homicide": 564,
            "larceny": 186401,
            "motor_vehicle_theft": 19909,
            "population": 11689442,
            "property_crime": 254496,
            "rape_legacy": null,
            "rape_revised": 5300,
            "robbery": 9185,
            "state_abbr": "OH",
            "state_id": 39,
            "violent_crime": 32723,
            "year": 2018
        }
    ]
}


In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#